In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re


In [2]:
page_number = 1
file_name = 'out-6-13-19.tsv'



#headers = 'index\tcommittee\tsubject\tdate\n'
#headers_list = ['index','committee','subject','date']

headers = 'index\tcommittee\tdate\tsubject\n'
headers_list = ['index','committee','date','subject']

In [3]:
for page in range(4):

    print(page_number)
    url = "http://dccouncil.us/events/list/?tribe_paged=%s"%str(page_number)

    res = requests.get(url)

    soup = bs(res.content,"lxml")

    calendar_date = soup.find("div",{'class':'listing-header__subhead'})
    articles = soup.find_all('article')
    ul_list = []
    hearing_dict = {}
    article_number = 0
    article_dict_list = []
    for article in articles:
        article_number +=1
        date = 'missing-date'
        committee = 'missing-committee'
        subject = 'missing-subject'


        date = article.time
        #date = re.match(r"<time datetime=\"(.*)\+0000",str(date))
        date = re.match(r"<time datetime=\"(.*)\-0400",str(date))
        try:
            date = date.group(1)
        except:
            date = 'NONETYPE'

        try:
            committee = article.a['title']
        except:
            committee = 'missing-committee'

        subj_list = []
        try:
            for item in article.ul:
                for item_2 in item:
                    if item_2 != '\n':
                        new_subj = item_2
                        '''
                        if '/Legislation/' in str(new_subj):
                            print('test' + str(new_subj))
                            new_subj = re.match(r"\</a>\,(.*)\</span\>",str(new_subj))
                            print(new_subj.group(1))
                            '''
                        subj_list.append(new_subj)
        except: subj_list = ['NoneType']

        subject = subj_list #article.ul

        hearing_dict={
            'index':article_number,
            'committee':committee,
            'subject':subject,
            'date':date}
        article_dict_list.append(hearing_dict)
        #print(article_dict_list)
        ul_list.append(article.ul)

1
1
1
1


In [34]:
test = article_dict_list[7]['subject']
test[0]

<span style="font-size: 11.0pt; font-family: 'Calibri',sans-serif;"><span style="font-size: 11.0pt; font-family: 'Calibri',sans-serif;"><a href="http://lims.dccouncil.us/Legislation/B23-0092">B23-92</a>, the “Grocery Store Development Projects Labor Peace Agreement Act of 2019” and</span></span>

In [45]:
stripped = re.match(r"<span.*;\">",str(test[0]))

In [127]:
stripped.group(0)

'<span style="font-size: 11.0pt; font-family: \'Calibri\',sans-serif;"><span style="font-size: 11.0pt; font-family: \'Calibri\',sans-serif;">'

In [55]:
a = str(test[0]).strip(str(stripped.group(0)))

In [108]:
tt = re.search(r"<a href=\"(.*)\">",str(test[0]))

In [125]:
tt.group(1)

'http://lims.dccouncil.us/Legislation/B23-0092'

In [122]:
t2 = re.search(r'{}(.*)</a>'.format(str(tt.group(0))),str(test[0]),re.MULTILINE)

In [124]:
t2.group(1)

'B23-92'

In [128]:
import date

ModuleNotFoundError: No module named 'date'

## Creating a workspace to transition to full html searching

In [174]:
all_hearings = []
previous_date = ''
with open('testHTMLOut.html','w') as f:
    f.write('')
    f.close()
    
for page_number in range(3):
    page_number +=1
    url = "http://dccouncil.us/events/list/?tribe_paged=%s"%str(page_number)

    res = requests.get(url)

    soup = bs(res.content,"lxml")

    calendar_date = soup.find("div",{'class':'listing-header__subhead'})

    articles = soup.find_all('article')
    ul_list = []
    hearing_dict = {}
    article_number = 0
    article_dict_list = []

    for article in articles:
        article_number +=1
        date = 'missing-date'
        committee = 'missing-committee'
        subject = 'missing-subject'


        date = article.time
        #date = re.match(r"<time datetime=\"(.*)\+0000",str(date))
        date = re.match(r"<time datetime=\"(.*)\-0400",str(date))
        try:
            date = date.group(1)
        except:
            date = 'NONETYPE'

        try:
            committee = article.a['title']
        except:
            committee = 'missing-committee'

        subj_list = []
        try:
            subj = article.find("div", {"class": "base"})
            subj_2 = subj.find("a", {"class" : "pdfemb-viewer"}).decompose()

            #subj_3 = subj.find("a", {"class" : "button-text-link"}).decompose()
            #subj_4 = re.find(r"\"]",str(subj))
            #pdf_viewer = subj.findAll("a", {"class": "pdfemb-viewer"})
            #button_text = subj.findAll("a", {"class": "button-text-link"})
        except: subj = ['NoneType']

        subject = subj #article.ul
        stripped_date = date[5:16]
        if stripped_date == previous_date: stripped_date = ''
        previous_date = date[5:16]
        
        hearing_dict={
            'index':article_number,
            'committee':committee,
            'subject':subject,
            'date':date,
            'date_to_show':stripped_date,
            'time':time}
       
        
        article_dict_list.append(hearing_dict)
        all_hearings.append(hearing_dict)
        #print(article_dict_list)
        head = "<head><link rel=\"stylesheet\" type=\"text/css\" href=\"testStyle.css\"></head> "
        date_header = "<h1>{}</h1>".format(hearing_dict["date_to_show"])
        committee_header = "<div class = \"hearingHeader\"> <h2>{}</h2>".format(hearing_dict["committee"])
        date_subheader = "<h3>{}</h3> </div>".format(hearing_dict["date"])
        subj_content = "{}".format(hearing_dict["subject"])

        html = "{}{}{}{}{}".format(head,date_header,committee_header,date_subheader,subj_content)
        with open('testHTMLOut.html','a') as f:
            f.write(html)
            f.close()


In [163]:
all_hearings[5]['date'][5:16]

'19 Jun 2019'

In [138]:
head = "<head><link rel=\"stylesheet\" type=\"text/css\" href=\"testStyle.css\"></head> "
committee_header = "<h2>{}</h2>".format(hearing_dict["committee"])
date_header = "<h1>{}</h1>".format(hearing_dict["date"])
subj_content = "{}".format(hearing_dict["subject"])

html = "{}{}{}{}".format(head,date_header,committee_header,subj_content)
with open('testHTMLOut.html','w') as f:
    f.write(html)
    f.close()

In [100]:
no_brack = re.findall(r"<(.*)/>",str(subj))

In [107]:
tag_list = []
tag_list = [tag.name for tag in subj.find_all()]

In [108]:
for tag in tag_list:
    no_brack = re.findall(r"<(.*)/>",str(subj))

['p', 'ul', 'li', 'span', 'a', 'li', 'span', 'a', 'p', 'a', 'p', 'p']

In [127]:
hearing_dict['date']

<div class="listing-header__subhead">Wednesday<br/>June 19</div>

In [146]:
hearing_dict

{'index': 1,
 'committee': 'missing-committee',
 'subject': ['NoneType'],
 'date': 'NONETYPE',
 'time': <time>All Day</time>}